In [1]:
!pip install openpyxl

In [2]:
import pandas as pd
import json
import boto3
from datetime import datetime, timedelta
import numpy as np


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
import configparser
## acess s3 data
config = configparser.ConfigParser()
config.read_file(open('/home/sandra/code/udacity/Project3_Redshift/dwh1.cfg'))

S3_UDACITY_REGION      = config.get("S3", "UDACITY_REGION")
S3_UDACITY_BUCKET      = config.get("S3", "UDACITY_BUCKET")
S3_MYBUCKET            = config.get("S3", "MY_S3")
DWH_ROLE_ARN           = config.get("ARN","ARN_RULE")
AWS_KEY                = config.get("AWS", "key")
AWS_SECRET             = config.get("AWS", "secret")


## -------------------------------- Inflation Rate (IPCA Brazil) -----------------------------------------

In [5]:
## there is month/year that can be formatted in the call

In [6]:
link = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/63/p/202112/d/v63%202"
df_ipca = pd.read_json(link)
df_ipca

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Mês (Código),Mês
1,1,Brasil,2,%,0.73,1,Brasil,63,IPCA - Variação mensal,202112,dezembro 2021


In [7]:
df_ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NC      2 non-null      object
 1   NN      2 non-null      object
 2   MC      2 non-null      object
 3   MN      2 non-null      object
 4   V       2 non-null      object
 5   D1C     2 non-null      object
 6   D1N     2 non-null      object
 7   D2C     2 non-null      object
 8   D2N     2 non-null      object
 9   D3C     2 non-null      object
 10  D3N     2 non-null      object
dtypes: object(11)
memory usage: 304.0+ bytes


## --------------------- Exchange rate USD (dolar) ---------------------------------------------

In [8]:
##there is month/year that can be formatted in the call

In [9]:
link = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?%40moeda=%27USD%27&%40dataInicial=%2701-31-2018%27&%40dataFinalCotacao=%2701-09-2022%27&%24format=json"
df_rate = pd.read_json(link)
df_rate = pd.json_normalize(df_rate['value'])
df_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4918 entries, 0 to 4917
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   paridadeCompra   4918 non-null   float64
 1   paridadeVenda    4918 non-null   float64
 2   cotacaoCompra    4918 non-null   float64
 3   cotacaoVenda     4918 non-null   float64
 4   dataHoraCotacao  4918 non-null   object 
 5   tipoBoletim      4918 non-null   object 
dtypes: float64(4), object(2)
memory usage: 230.7+ KB


In [10]:
df_rate

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,1.0,1.0,3.1513,3.1519,2018-01-31 10:05:22.398,Abertura
1,1.0,1.0,3.1567,3.1573,2018-01-31 11:12:39.258,Intermediário
2,1.0,1.0,3.1757,3.1763,2018-01-31 12:10:21.761,Intermediário
3,1.0,1.0,3.1633,3.1639,2018-01-31 13:16:53.496,Intermediário
4,1.0,1.0,3.1618,3.1624,2018-01-31 13:16:53.521,Fechamento
...,...,...,...,...,...,...
4913,1.0,1.0,5.6906,5.6912,2022-01-07 10:06:19.793,Abertura
4914,1.0,1.0,5.7032,5.7038,2022-01-07 11:06:20.501,Intermediário
4915,1.0,1.0,5.6488,5.6494,2022-01-07 12:10:20.558,Intermediário
4916,1.0,1.0,5.6563,5.6569,2022-01-07 13:02:38.2,Intermediário


In [11]:
df_rate.shape

(4918, 6)

## --------------------- Price rate Brent crude oil ------------------------------

In [12]:
## single access - there is no option to format month/year

In [13]:
link =  "http://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='EIA366_PBRENT366')"
df_brent = pd.read_json(link)

df_brent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13043 entries, 0 to 13042
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   @odata.context  13043 non-null  object
 1   value           13043 non-null  object
dtypes: object(2)
memory usage: 203.9+ KB


In [14]:
df_brent

,@odata.context,value
0,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
1,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
2,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
3,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
4,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
...,...,...
13038,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
13039,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
13040,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."
13041,http://ipeadata.gov.br/api/odata4/$metadata#Co...,"{'SERCODIGO': 'EIA366_PBRENT366', 'VALDATA': '..."


In [15]:
df_brent = pd.read_json(link)
df_brent = pd.json_normalize(df_brent['value'])
df_brent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13043 entries, 0 to 13042
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SERCODIGO  13043 non-null  object 
 1   VALDATA    13043 non-null  object 
 2   VALVALOR   10773 non-null  float64
 3   NIVNOME    13043 non-null  object 
 4   TERCODIGO  13043 non-null  object 
dtypes: float64(1), object(4)
memory usage: 509.6+ KB


In [16]:
df_brent.tail(3)

,SERCODIGO,VALDATA,VALVALOR,NIVNOME,TERCODIGO
13040,EIA366_PBRENT366,2022-10-13T00:00:00-03:00,95.16,,
13041,EIA366_PBRENT366,2022-10-14T00:00:00-03:00,92.22,,
13042,EIA366_PBRENT366,2022-10-17T00:00:00-03:00,91.04,,


## ------------ Fuel price series - historical series up to previous year -----

In [17]:
## first time until 2021 and can be formatted by semester in csv download
## 2004/01 => first half of 2004
## 2004/02 => second half of 2004

In [18]:
link1 = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2018-01.csv"
df_hist = pd.read_csv(link1, sep=";", decimal=",", encoding='latin-1')


In [19]:
df_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469833 entries, 0 to 469832
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ï»¿Regiao - Sigla  469833 non-null  object 
 1   Estado - Sigla     469833 non-null  object 
 2   Municipio          469833 non-null  object 
 3   Revenda            469833 non-null  object 
 4   CNPJ da Revenda    469833 non-null  object 
 5   Nome da Rua        469833 non-null  object 
 6   Numero Rua         469485 non-null  object 
 7   Complemento        123062 non-null  object 
 8   Bairro             468412 non-null  object 
 9   Cep                469833 non-null  object 
 10  Produto            469833 non-null  object 
 11  Data da Coleta     469833 non-null  object 
 12  Valor de Venda     469833 non-null  float64
 13  Valor de Compra    188187 non-null  float64
 14  Unidade de Medida  469833 non-null  object 
 15  Bandeira           469833 non-null  object 
dtypes:

In [50]:
df_hist.columns

Index(['ï»¿Regiao - Sigla', 'Estado - Sigla', 'Municipio', 'Revenda',
       'CNPJ da Revenda', 'Nome da Rua', 'Numero Rua', 'Complemento', 'Bairro',
       'Cep', 'Produto', 'Data da Coleta', 'Valor de Venda', 'Valor de Compra',
       'Unidade de Medida', 'Bandeira'],
      dtype='object')

In [21]:
df_hist[df_hist.Municipio.str.contains('PAULO')]

,ï»¿Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
7670,SE,SP,SAO PAULO,AUTO POSTO GELEIÃO II LTDA,09.562.368/0001-18,RUA ALTO BELO,850,NaN,VILA ANTONIETA,03478-040,DIESEL S10,02/01/2018,3.299,NaN,R$ / litro,BRANCA
7671,SE,SP,SAO PAULO,AUTO POSTO GELEIÃO II LTDA,09.562.368/0001-18,RUA ALTO BELO,850,NaN,VILA ANTONIETA,03478-040,DIESEL,02/01/2018,3.199,NaN,R$ / litro,BRANCA
7672,SE,SP,SAO PAULO,AUTO POSTO GELEIÃO II LTDA,09.562.368/0001-18,RUA ALTO BELO,850,NaN,VILA ANTONIETA,03478-040,ETANOL,02/01/2018,2.699,NaN,R$ / litro,BRANCA
7673,SE,SP,SAO PAULO,AUTO POSTO GELEIÃO II LTDA,09.562.368/0001-18,RUA ALTO BELO,850,NaN,VILA ANTONIETA,03478-040,GASOLINA,02/01/2018,3.799,NaN,R$ / litro,BRANCA
7674,SE,SP,SAO PAULO,AUTO POSTO RENOVACAO LTDA,09.258.341/0001-36,AVENIDA VILA EMA,3745,NaN,VILA EMA,03281-001,ETANOL,04/01/2018,2.699,2.5192,R$ / litro,BRANCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469341,SE,SP,SAO PAULO,AUTO POSTO MONET LTDA,20.710.409/0001-09,RUA MARIA FETTE,161,NaN,VILA MERCEDES,03263-000,ETANOL,26/06/2018,2.499,2.1799,R$ / litro,BRANCA
469394,SE,SP,SAO PAULO,POSTO DE SERVIÃOS MALTA 02 LTDA,20.893.966/0001-02,RUA ROQUE BARBOSA LIMA,326,NaN,VILA PAULO SILAS,03264-040,GASOLINA,26/06/2018,4.199,3.8162,R$ / litro,RAIZEN
469395,SE,SP,SAO PAULO,POSTO DE SERVIÃOS MALTA 02 LTDA,20.893.966/0001-02,RUA ROQUE BARBOSA LIMA,326,NaN,VILA PAULO SILAS,03264-040,ETANOL,26/06/2018,2.599,2.4796,R$ / litro,RAIZEN
469806,SE,SP,SAO PAULO,CENTRO AUTOMOTIVO PORTO PRINCIPE LTDA,23.226.275/0001-43,AVENIDA PRESIDENTE TANCREDO NEVES,126,142,VILA NANCY,04287-010,GASOLINA,25/06/2018,4.049,3.6750,R$ / litro,BRANCA


## ------------ Fuel price series - current year -----

In [22]:
## from 2021 and for current year - format ano/mes
## 2022/precos-gasolina-etanol-01 => janeiro current year

In [23]:
link0 = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/"
link1 = "shpc/dsan/2022/precos-gasolina-etanol-01.csv"
link = link0 + link1
link

headers = {"User-Agent": "pandas"}

df_atual = pd.read_csv(link, sep=";", storage_options=headers, decimal=',')
df_atual.tail(30)



,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
52519,SE,SP,PARAGUACU PAULISTA,AUTO POSTO KAWAKAMI LTDA,28.973.159/0003-73,RUA PREF JOSE DELIBERADOR,323,SALA 1,CENTRO,19700-000,GASOLINA,31/01/2022,6.140,NaN,R$ / litro,BRANCA
52520,SE,SP,PARAGUACU PAULISTA,AUTO POSTO KAWAKAMI LTDA,28.973.159/0003-73,RUA PREF JOSE DELIBERADOR,323,SALA 1,CENTRO,19700-000,ETANOL,31/01/2022,4.690,NaN,R$ / litro,BRANCA
52521,SE,SP,PARAGUACU PAULISTA,AUTO POSTO KAWAKAMI LTDA,28.973.159/0003-73,RUA PREF JOSE DELIBERADOR,323,SALA 1,CENTRO,19700-000,GASOLINA ADITIVADA,31/01/2022,6.150,NaN,R$ / litro,BRANCA
52522,SE,SP,LINS,AUTO POSTO PERNAMBUCO DE LINS LTDA,31.750.836/0001-37,RUA PERNAMBUCO,450,NaN,VILA SAO JOSE,16401-047,GASOLINA,31/01/2022,6.350,NaN,R$ / litro,BRANCA
52523,SE,SP,LINS,AUTO POSTO PERNAMBUCO DE LINS LTDA,31.750.836/0001-37,RUA PERNAMBUCO,450,NaN,VILA SAO JOSE,16401-047,ETANOL,31/01/2022,4.650,NaN,R$ / litro,BRANCA
52524,SE,SP,LINS,AUTO POSTO PERNAMBUCO DE LINS LTDA,31.750.836/0001-37,RUA PERNAMBUCO,450,NaN,VILA SAO JOSE,16401-047,GASOLINA ADITIVADA,31/01/2022,6.350,NaN,R$ / litro,BRANCA
52525,SE,SP,SAO BERNARDO DO CAMPO,STAR DUST AUTO POSTO LTDA,25.014.492/0001-03,AVENIDA MARIA SERVIDEI DEMARCHI,1661,NaN,DEMARCHI,09820-000,GASOLINA,31/01/2022,6.177,NaN,R$ / litro,BRANCA
52526,SE,SP,SAO BERNARDO DO CAMPO,STAR DUST AUTO POSTO LTDA,25.014.492/0001-03,AVENIDA MARIA SERVIDEI DEMARCHI,1661,NaN,DEMARCHI,09820-000,ETANOL,31/01/2022,4.677,NaN,R$ / litro,BRANCA
52527,SE,SP,SAO BERNARDO DO CAMPO,STAR DUST AUTO POSTO LTDA,25.014.492/0001-03,AVENIDA MARIA SERVIDEI DEMARCHI,1661,NaN,DEMARCHI,09820-000,GASOLINA ADITIVADA,31/01/2022,6.277,NaN,R$ / litro,BRANCA
52528,NE,BA,SALVADOR,FCL COMERCIO DE COMBUSTIVEIS LTDA.,29.456.610/0001-86,ESTRADA DO COQUEIRO GRANDE,931,NaN,FAZENDA GRANDE III,41343-855,GASOLINA,31/01/2022,6.830,NaN,R$ / litro,BRANCA


In [24]:
link0 = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/"
link1 = "shpc/dsan/2022/precos-gasolina-etanol-08.csv"
link = link0 + link1
link


'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsan/2022/precos-gasolina-etanol-08.csv'

In [25]:
df_atual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52549 entries, 0 to 52548
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Regiao - Sigla     52549 non-null  object 
 1   Estado - Sigla     52549 non-null  object 
 2   Municipio          52549 non-null  object 
 3   Revenda            52549 non-null  object 
 4   CNPJ da Revenda    52549 non-null  object 
 5   Nome da Rua        52549 non-null  object 
 6   Numero Rua         52528 non-null  object 
 7   Complemento        10994 non-null  object 
 8   Bairro             52411 non-null  object 
 9   Cep                52549 non-null  object 
 10  Produto            52549 non-null  object 
 11  Data da Coleta     52549 non-null  object 
 12  Valor de Venda     52549 non-null  float64
 13  Valor de Compra    0 non-null      float64
 14  Unidade de Medida  52549 non-null  object 
 15  Bandeira           52549 non-null  object 
dtypes: float64(2), object(

## ---------- Data from Brazilian municipalities ----------------

In [26]:
link = "http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista-de-Munic%C3%ADpios-com-IBGE-Brasil.xlsx"

df_city = pd.read_excel(link)
df_city.head()

,ConcatUF+Mun,IBGE,IBGE7,UF,Município,Região,População 2010,Porte,Capital,Unnamed: 9
0,ROAlta Floresta D'oeste,110001,1100015,RO,Alta Floresta D´oeste,Região Norte,24392.0,Pequeno II,NaN,PROCV(A2;'[Lista de Municípios com IBGE - Bras...
1,ROAriquemes,110002,1100023,RO,Ariquemes,Região Norte,90353.0,Médio,NaN,PROCV(A2;'Lista de Municípios IBGE'!$A$2:$G$55...
2,ROCabixi,110003,1100031,RO,Cabixi,Região Norte,6313.0,Pequeno I,NaN,NaN
3,ROCacoal,110004,1100049,RO,Cacoal,Região Norte,78574.0,Médio,NaN,NaN
4,ROCerejeiras,110005,1100056,RO,Cerejeiras,Região Norte,17029.0,Pequeno I,NaN,NaN


In [27]:
df_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ConcatUF+Mun    5570 non-null   object 
 1   IBGE            5570 non-null   int64  
 2   IBGE7           5570 non-null   int64  
 3   UF              5570 non-null   object 
 4   Município       5570 non-null   object 
 5   Região          5570 non-null   object 
 6   População 2010  5565 non-null   float64
 7   Porte           5570 non-null   object 
 8   Capital         27 non-null     object 
 9   Unnamed: 9      2 non-null      object 
dtypes: float64(1), int64(2), object(7)
memory usage: 435.3+ KB


In [28]:
df_city['cidade'] = df_city['Município'].str.upper()
df_city.columns


Index(['ConcatUF+Mun', 'IBGE', 'IBGE7', 'UF', 'Município', 'Região',
       'População 2010', 'Porte', 'Capital', 'Unnamed: 9', 'cidade'],
      dtype='object')

In [29]:
link = "http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista-de-Munic%C3%ADpios-com-IBGE-Brasil.xlsx"

df_ct = pd.read_excel(link, skiprows=1, header=None)



In [30]:
df_ct.rename(columns={0: 'uf_mun', 1: 'ibge', 2: 'ibge7', 3: 'uf', 4: 'municipio', 5:'regiao',
       6:'populacao_2010', 7:'porte', 8: 'capital', 9: 'unnamed_9', 10: 'cidade'}, inplace=True)

In [31]:
cols = df_ct.select_dtypes(include=[np.object]).columns
df_ct[cols] = df_ct[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df_ct


<ipython-input-31-0a0d9bc19c55>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = df_ct.select_dtypes(include=[np.object]).columns


,uf_mun,ibge,ibge7,uf,municipio,regiao,populacao_2010,porte,capital,unnamed_9
0,ROAlta Floresta D'oeste,110001,1100015,RO,Alta Floresta D oeste,Regiao Norte,24392.0,Pequeno II,NaN,PROCV(A2;'[Lista de Municipios com IBGE - Bras...
1,ROAriquemes,110002,1100023,RO,Ariquemes,Regiao Norte,90353.0,Medio,NaN,PROCV(A2;'Lista de Municipios IBGE'!$A$2:$G$55...
2,ROCabixi,110003,1100031,RO,Cabixi,Regiao Norte,6313.0,Pequeno I,NaN,NaN
3,ROCacoal,110004,1100049,RO,Cacoal,Regiao Norte,78574.0,Medio,NaN,NaN
4,ROCerejeiras,110005,1100056,RO,Cerejeiras,Regiao Norte,17029.0,Pequeno I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5565,GOVianopolis,522200,5222005,GO,Vianopolis,Regiao Centro-Oeste,12548.0,Pequeno I,NaN,NaN
5566,GOVicentinopolis,522205,5222054,GO,Vicentinopolis,Regiao Centro-Oeste,7371.0,Pequeno I,NaN,NaN
5567,GOVila Boa,522220,5222203,GO,Vila Boa,Regiao Centro-Oeste,4735.0,Pequeno I,NaN,NaN
5568,GOVila Propicio,522230,5222302,GO,Vila Propicio,Regiao Centro-Oeste,5145.0,Pequeno I,NaN,NaN


## ---------- Minimum Wage - value series ----------------

In [32]:
link =  "http://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='MTE12_SALMIN12')"
df_minimo = pd.read_json(link)
df_minimo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   @odata.context  990 non-null    object
 1   value           990 non-null    object
dtypes: object(2)
memory usage: 15.6+ KB


In [53]:
df_minimo1 = pd.read_json(link)
df_minimo1 = pd.json_normalize(df_minimo['value'])
df_minimo1

,SERCODIGO,VALDATA,VALVALOR,NIVNOME,TERCODIGO
0,MTE12_SALMIN12,1940-07-01T00:00:00-03:00,0.00000,,
1,MTE12_SALMIN12,1940-08-01T00:00:00-03:00,0.00000,,
2,MTE12_SALMIN12,1940-09-01T00:00:00-03:00,0.00000,,
3,MTE12_SALMIN12,1940-10-01T00:00:00-03:00,0.00000,,
4,MTE12_SALMIN12,1940-11-01T00:00:00-03:00,0.00000,,
...,...,...,...,...,...
985,MTE12_SALMIN12,2022-08-01T00:00:00-03:00,1212.00000,,
986,MTE12_SALMIN12,2022-09-01T00:00:00-03:00,1212.00000,,
987,MTE12_SALMIN12,2022-10-01T00:00:00-03:00,1212.00000,,
988,MTE12_SALMIN12,2022-11-01T00:00:00-03:00,1212.00000,,


In [34]:
pd.options.display.float_format = '{:.5f}'.format
df_minimo1

,SERCODIGO,VALDATA,VALVALOR,NIVNOME,TERCODIGO
0,MTE12_SALMIN12,1940-07-01T00:00:00-03:00,0.00000,,
1,MTE12_SALMIN12,1940-08-01T00:00:00-03:00,0.00000,,
2,MTE12_SALMIN12,1940-09-01T00:00:00-03:00,0.00000,,
3,MTE12_SALMIN12,1940-10-01T00:00:00-03:00,0.00000,,
4,MTE12_SALMIN12,1940-11-01T00:00:00-03:00,0.00000,,
...,...,...,...,...,...
985,MTE12_SALMIN12,2022-08-01T00:00:00-03:00,1212.00000,,
986,MTE12_SALMIN12,2022-09-01T00:00:00-03:00,1212.00000,,
987,MTE12_SALMIN12,2022-10-01T00:00:00-03:00,1212.00000,,
988,MTE12_SALMIN12,2022-11-01T00:00:00-03:00,1212.00000,,


##      ------------- write file into S3 ----------------------------

In [35]:
#Creating Session With Boto3.
session = boto3.Session(
aws_access_key_id=AWS_KEY,
aws_secret_access_key=AWS_SECRET)

session

Session(region_name='us-west-2')

## parsing each dataset

### City and state

In [36]:
df_city.columns

Index(['ConcatUF+Mun', 'IBGE', 'IBGE7', 'UF', 'Município', 'Região',
       'População 2010', 'Porte', 'Capital', 'Unnamed: 9', 'cidade'],
      dtype='object')

In [37]:
df_city['Região'].value_counts()

Região Nordeste        1794
Região Sudeste         1668
Região Sul             1191
Região Centro-Oeste     467
Região Norte            450
Name: Região, dtype: int64

In [38]:
df_city['Porte'].value_counts()
# pequeno I  = small I
# pequeno II = small II
# medio      = midsize
# grande    = large
# metropole = metropolis


Pequeno I     3919
Pequeno II    1043
Médio          325
Grande         266
Metrópole       17
Name: Porte, dtype: int64

### Infaltion rate (IPCA)

In [39]:
df_ipca.columns

Index(['NC', 'NN', 'MC', 'MN', 'V', 'D1C', 'D1N', 'D2C', 'D2N', 'D3C', 'D3N'], dtype='object')

In [40]:
df_ipca.head(5)

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Mês (Código),Mês
1,1,Brasil,2,%,0.73,1,Brasil,63,IPCA - Variação mensal,202112,dezembro 2021


In [41]:
df_ipca['D2N'].value_counts()

Variável                  1
IPCA - Variação mensal    1
Name: D2N, dtype: int64

### USD exchange rate 

In [42]:
df_rate.head(5)

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,1.00000,1.00000,3.15130,3.15190,2018-01-31 10:05:22.398,Abertura
1,1.00000,1.00000,3.15670,3.15730,2018-01-31 11:12:39.258,Intermediário
2,1.00000,1.00000,3.17570,3.17630,2018-01-31 12:10:21.761,Intermediário
3,1.00000,1.00000,3.16330,3.16390,2018-01-31 13:16:53.496,Intermediário
4,1.00000,1.00000,3.16180,3.16240,2018-01-31 13:16:53.521,Fechamento


In [43]:
df_rate.columns

Index(['paridadeCompra', 'paridadeVenda', 'cotacaoCompra', 'cotacaoVenda',
       'dataHoraCotacao', 'tipoBoletim'],
      dtype='object')

In [44]:
df_rate.tipoBoletim.value_counts()

Intermediário    2940
Abertura          989
Fechamento        989
Name: tipoBoletim, dtype: int64

### Price rate Brent crude oil 

In [45]:
df_brent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13043 entries, 0 to 13042
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SERCODIGO  13043 non-null  object 
 1   VALDATA    13043 non-null  object 
 2   VALVALOR   10773 non-null  float64
 3   NIVNOME    13043 non-null  object 
 4   TERCODIGO  13043 non-null  object 
dtypes: float64(1), object(4)
memory usage: 509.6+ KB


In [46]:
df_brent[(df_brent.VALDATA.str.slice(0,4,1) == '2019') & (~df_brent.VALVALOR.isnull())].head(10)

,SERCODIGO,VALDATA,VALVALOR,NIVNOME,TERCODIGO
12049,EIA366_PBRENT366,2019-01-02T00:00:00-02:00,54.06000,,
12050,EIA366_PBRENT366,2019-01-03T00:00:00-02:00,54.06000,,
12051,EIA366_PBRENT366,2019-01-04T00:00:00-02:00,53.23000,,
12052,EIA366_PBRENT366,2019-01-05T00:00:00-02:00,55.64000,,
12054,EIA366_PBRENT366,2019-01-07T00:00:00-02:00,57.10000,,
12055,EIA366_PBRENT366,2019-01-08T00:00:00-02:00,57.10000,,
12056,EIA366_PBRENT366,2019-01-09T00:00:00-02:00,56.91000,,
12057,EIA366_PBRENT366,2019-01-10T00:00:00-02:00,59.46000,,
12058,EIA366_PBRENT366,2019-01-11T00:00:00-02:00,60.47000,,
12059,EIA366_PBRENT366,2019-01-12T00:00:00-02:00,59.24000,,


In [47]:
df_brent.columns

Index(['SERCODIGO', 'VALDATA', 'VALVALOR', 'NIVNOME', 'TERCODIGO'], dtype='object')

### Minimum wage

In [54]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [55]:
df_minimo1

,SERCODIGO,VALDATA,VALVALOR,NIVNOME,TERCODIGO
0,MTE12_SALMIN12,1940-07-01T00:00:00-03:00,0.00000,,
1,MTE12_SALMIN12,1940-08-01T00:00:00-03:00,0.00000,,
2,MTE12_SALMIN12,1940-09-01T00:00:00-03:00,0.00000,,
3,MTE12_SALMIN12,1940-10-01T00:00:00-03:00,0.00000,,
4,MTE12_SALMIN12,1940-11-01T00:00:00-03:00,0.00000,,
...,...,...,...,...,...
985,MTE12_SALMIN12,2022-08-01T00:00:00-03:00,1212.00000,,
986,MTE12_SALMIN12,2022-09-01T00:00:00-03:00,1212.00000,,
987,MTE12_SALMIN12,2022-10-01T00:00:00-03:00,1212.00000,,
988,MTE12_SALMIN12,2022-11-01T00:00:00-03:00,1212.00000,,


In [56]:
pd.reset_option('display.float_format')

### fuel price history 

In [57]:
df_hist.head()

,ï»¿Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,GNV,02/01/2018,2.370,1.7383,R$ / mÂ³,PETROBRAS DISTRIBUIDORA S.A.
1,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,DIESEL S10,02/01/2018,3.240,3.1366,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
2,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,ETANOL,02/01/2018,2.930,2.6194,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
3,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,GASOLINA,02/01/2018,3.620,3.6017,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
4,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,GNV,02/01/2018,2.699,NaN,R$ / mÂ³,BRANCA


In [58]:
df_hist[df_hist.Produto == 'GASOLINA']['Unidade de Medida'].value_counts()

R$ / litro    144051
Name: Unidade de Medida, dtype: int64

In [ ]:
df_hist.columns

In [59]:
## select rows: Product = Gasolin(gasoline) and fuel distributor = Petrobras
## get year and month from period
## select relevant columns

def fuel(df_hist, df_state):
    df_hist['dates'] = pd.to_datetime(df_hist['Data da Coleta'], format='%d/%m/%Y')
    df_hist['year_month'] = pd.to_datetime(df_hist['dates']).dt.to_period('M').astype(str)
        
    new_name = {'Estado - Sigla': 'UF', 'Valor de Venda': 'Venda', 'Valor de Compra': 'Compra'}
    df_hist.rename(columns=new_name, inplace=True)

    df_sel = df_hist[["UF",'Municipio','year_month', 'Produto', 'Compra', 'Venda']]

    df_mean = df_sel.groupby(['UF', 'Municipio', 'year_month', 'Produto']).mean().reset_index()
    df_merge = pd.merge(df_mean, df_city, how='inner', left_on=['Municipio','UF'], right_on=['cidade', 'UF'])

    #                 Sales Value R$20.000  |   Purchase Value: R$13.000
    #Profit:          Sales Value R$20.000 - Purchase Value R$13.000 = Profit R$7.000
    #Margin in %      Profit R$7.000 / Sales Value R$20.000 = 0.35 x 100 = 35%
    
    df_merge['Profit %'] = ((df_merge.Venda - df_merge.Compra) / df_merge.Venda) * 100
    df_merge =  df_merge[['IBGE', 'UF', 'Municipio', 'Produto','year_month', 'Compra','Venda', 'Capital', 'Profit %']]
    df_merge.sort_values(by=['IBGE', 'UF', 'Municipio', 'Produto','year_month'], inplace=True)
    return df_merge

In [60]:
df_merge = fuel(df_hist, df_city)
df_merge.info()
df_merge

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6947 entries, 4142 to 1313
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IBGE        6947 non-null   int64  
 1   UF          6947 non-null   object 
 2   Municipio   6947 non-null   object 
 3   Produto     6947 non-null   object 
 4   year_month  6947 non-null   object 
 5   Compra      6295 non-null   float64
 6   Venda       6947 non-null   float64
 7   Capital     438 non-null    object 
 8   Profit %    6295 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 542.7+ KB


,IBGE,UF,Municipio,Produto,year_month,Compra,Venda,Capital,Profit %
4142,110002,RO,ARIQUEMES,DIESEL,2018-01,3.175033,3.649122,NaN,12.991855
4146,110002,RO,ARIQUEMES,DIESEL,2018-02,3.143900,3.640469,NaN,13.640242
4150,110002,RO,ARIQUEMES,DIESEL,2018-03,3.145014,3.640312,NaN,13.605926
4154,110002,RO,ARIQUEMES,DIESEL,2018-04,3.216789,3.668375,NaN,12.310249
4158,110002,RO,ARIQUEMES,DIESEL,2018-05,3.398250,3.886333,NaN,12.558967
...,...,...,...,...,...,...,...,...,...
1297,522140,GO,TRINDADE,GASOLINA,2018-02,3.860622,4.360321,NaN,11.460146
1301,522140,GO,TRINDADE,GASOLINA,2018-03,3.842300,4.357679,NaN,11.826907
1305,522140,GO,TRINDADE,GASOLINA,2018-04,3.880888,4.377929,NaN,11.353339
1309,522140,GO,TRINDADE,GASOLINA,2018-05,3.910380,4.403138,NaN,11.191063


### current fuel price

In [61]:
# Petrobras distribuitor changed to Vibra Energia from Ago/2020
# the same consideration for historical fuel price

In [62]:
df_fuel = fuel(df_atual, df_city)
df_fuel[df_fuel.Venda.isnull()]

,IBGE,UF,Municipio,Produto,year_month,Compra,Venda,Capital,Profit %


In [63]:
df_fuel.shape

(687, 9)

## ---- Project Objective ----

## Get the graph of fuel price timeline

![drawing](graph_fuel_timeline.jpg)